Joseph D. Piotroski, "Value Investing: The Use of Historical Financial Statement Information to Separate Winners from Losers" (2000); sugere uma estratégia de avaliação da saúde financeira de uma empresa. A avaliação consiste na análise de 9 critérios que somados levam a uma pontuação final denominada Piotroski F-Score. Os critérios a serem avaliados e regras de pontuação são os seguintes:
- Retorno sobre Ativos (ROA): Lucro Líquido Anual / Ativos Totais; indica a capacidade da empresa de gerar lucro a partir dos seus ativos. Caso seja positivo, somar 1 à pontuação.
- Variação do ROA: Caso o ROA do ano corrente seja maior que o ROA do ano anterior, somar 1 à pontuação.
- Fluxo de Caixa Operacional (FCO): o resultado de uma empresa considerando apenas as movimentações operacionais, descartando efeitos puramente financeiro ou contábil, caso seja positivo para o ano corrente, somar 1 à pontuação.
- FCO > Lucro Líquido (LL): lucro líquido maior do que o caixa gerado por suas operações pode indicar truques contábeis que tendem a gerar perda de valor para o acionista no futuro. Caso o FCO seja maior do que o LL para o período analisado, somar 1 à pontuação.
- Variação na Alavancagem: endividamento em excesso é uma causa comum de destruição de valor ao longo do tempo. Caso a alavancagem da empresa tenha diminuído com relação ao período anterior, somar 1 à pontuação.
- Variação na Liquidez Corrente: Ativo Circulante / Passivo Circulante e serve para indicar a capacidade da empresa de cumprir com as suas obrigações ao longo dos próximos 12 meses. Caso a liquidez corrente tenha aumentado em relação ao período anterior, somar 1 à pontuação.
- Variação no Total de Ações: Em geral, não é interessante para o acionista que a empresa emita novas ações constantemente, caso a empresa não tenha aumentado o número de ações em circulação no último ano, somar 1 à pontuação.
- Variação na Margem Bruta: margem bruta (Lucro Bruto / Receita Líquida) elevada costumam indicar vantagem competitiva importante, pois este indicador demonstra a capacidade que a empresa possui de precificar seus produtos ou serviços acima dos custos de produção. Caso a margem bruta tenha aumentado em relação ao período anterior, somar 1 à pontuação.
- Variação no Giro do Ativo: Receita Líquida / Média dos Ativos Totais (últimos 12 meses). Através desse indicador, é possível identificar o quanto de receita a empresa consegue gerar a partir de seus ativos. Caso tenha aumentado em relação ao período anterior, somar 1 à pontuação.

O objetivo dessa análise é valorizar as empresas que possuam boas métricas de lucratividade e eficiência, além de demonstrarem estar evoluindo ao longo do tempo. No final, cada empresa terá uma pontuação entre 0 e 9, sendo que quanto maior a pontuação, melhor a situação financeira da mesma.

No estudo original de Piotroski, ao longo de 20 anos (1976–1996), uma estratégia de investimento baseado nessa pontuação, com a compra de empresas com F-Score alto e a venda de empresas com F-Score baixo, gerou um retorno anual de 23%, bem superior à media do mercado.

https://www.investopedia.com/terms/p/piotroski-score.asp

https://www.ivey.uwo.ca/media/3775523/value_investing_the_use_of_historical_financial_statement_information.pdf

In [1]:
import yfinance as yf
import pandas as pd
from IPython.display import display, HTML
import requests
import numpy as np

def b_print(df): 
    
    display(HTML(df.to_html(index=True)))
    df = None

info = [
'Net Income',           #Lucro Líquido
'Operating Cash Flow',  #Fluxo de Caixa Operacional
'Total Debt',           #Dívida Líquida
'Stockholders Equity',  #Patrimônio Líquido
'Current Assets',       #Ativo Circulante
'Current Liabilities',  #Passivo Circulante
'Gross Profit',         #Lucro Bruto
'Total Revenue',        #Receita Total
'Total Assets']         #Ativo Total


In [2]:
def piotroski_score(data):
  """
  Calcula o score de Piotroski para uma empresa com base em um DataFrame com dados do yahoo finance

  Args:
    data: DataFrame com index=['Net Income','Operating Cash Flow', etc.] e colunas=['2023', '2022'].

  Returns:
    int: Score de Piotroski (0 a 9).
  """

  score = 0

  # Rentabilidade
  score += int(data.loc['Net Income', '2023-12-31'] > 0)  # Lucro Líquido positivo
  score += int(data.loc['Operating Cash Flow', '2023-12-31'] > 0) # Fluxo de Caixa Operacional positivo
  score += int(data.loc['Net Income', '2023-12-31'] > data.loc['Net Income', '2022-12-31'])  # ROA superior ao ano anterior (aproximação) 
  score += int(data.loc['Operating Cash Flow', '2023-12-31'] > data.loc['Net Income', '2023-12-31'])  # Fluxo de Caixa Operacional superior ao Lucro Líquido

  # Alavancagem e Liquidez
  divida_patrimonio_2023 = data.loc['Total Debt', '2023-12-31'] / data.loc['Stockholders Equity', '2023-12-31']
  divida_patrimonio_2022 = data.loc['Total Debt', '2022-12-31'] / data.loc['Stockholders Equity', '2022-12-31']
  score += int(divida_patrimonio_2023 < divida_patrimonio_2022)  # Dívida/Ativo inferior ao ano anterior
  
  liquidez_corrente_2023 = data.loc['Current Assets', '2023-12-31'] / data.loc['Current Liabilities', '2023-12-31']
  liquidez_corrente_2022 = data.loc['Current Assets', '2022-12-31'] / data.loc['Current Liabilities', '2022-12-31']
  score += int(liquidez_corrente_2023 > liquidez_corrente_2022) # Liquidez corrente superior ao ano anterior
  # Emissões de ações: Não disponível no Yahoo Finance

  # Eficiência Operacional
  margem_bruta_2023 = data.loc['Gross Profit', '2023-12-31'] / data.loc['Total Revenue', '2023-12-31']
  margem_bruta_2022 = data.loc['Gross Profit', '2022-12-31'] / data.loc['Total Revenue', '2022-12-31']
  score += int(margem_bruta_2023 > margem_bruta_2022) # Margem bruta superior ao ano anterior

  giro_ativo_2023 = data.loc['Total Revenue', '2023-12-31'] / data.loc['Total Assets', '2023-12-31']
  giro_ativo_2022 = data.loc['Total Revenue', '2022-12-31'] / data.loc['Total Assets', '2022-12-31']
  score += int(giro_ativo_2023 > giro_ativo_2022)  # Giro do ativo superior ao ano anterior

  return score

In [4]:
import pyarrow.parquet as pq
# Abrir o arquivo Parquet
arquivo_parquet = pq.ParquetFile("data/yf_financial_data_Y.parquet")
tabela_parquet = arquivo_parquet.read()
df = tabela_parquet.to_pandas()
df

AALR3  \
                                                            2023   
Tax Effect Of Unusual Items                         4.432920e+06   
Tax Rate For Calcs                                  3.400000e-01   
Net Income From Continuing Operation Net Minori... -2.278780e+08   
Net Interest Income                                -2.491660e+08   
Interest Expense                                    2.028120e+08   
Interest Income                                     1.284000e+06   
Normalized Income                                  -2.364831e+08   
Net Income From Continuing And Discontinued Ope... -2.278780e+08   
Diluted NI Availto Com Stockholders                -2.278780e+08   
Net Income Common Stockholders                     -2.278780e+08   
Net Income                                         -2.278780e+08   
Net Income Including Noncontrolling Interests      -2.185590e+08   
Net Income Continuous Operations                   -2.185590e+08   
Pretax Income                                      -1.994690e+08   
Selling General And Administration                  3.250770e+08   
Total Revenue                                       1.179584e+09   
Operating Revenue                                   1.179584e+09   
Free Cash Flow                                      1.078400e+07   
End Cash Position                                   2.185950e+08   
Beginning Cash Position                             2.187440e+08   
Changes In Cash                                    -1.490000e+05   
Financing Cash Flow                                 8.697000e+06   
Investing Cash Flow                                -1.111500e+08   
Operating Cash Flow                                 1.023040e+08   
Change In Working Capital                           1.169840e+08   
Net Income From Continuing Operations              -2.185590e+08   
Ordinary Shares Number                              1.180805e+08   
Share Issued                                        1.182928e+08   
Tangible Book Value                                -8.457000e+07   
Net Tangible Assets                                -8.457000e+07   
Common Stock Equity                                 9.174560e+08   
Total Equity Gross Minority Interest                9.439920e+08   
Stockholders Equity                                 9.174560e+08   
Capital Stock                                       6.124120e+08   
Common Stock                                        6.124120e+08   
Total Liabilities Net Minority Interest             1.725819e+09   
Payables                                            2.482290e+08   
Total Assets                                        2.669811e+09   
Cash And Cash Equivalents                           2.185950e+08   

                                                                  \
                                                            2022   
Tax Effect Of Unusual Items                        -1.152656e+06   
Tax Rate For Calcs                                  6.100000e-02   
Net Income From Continuing Operation Net Minori... -2.278100e+08   
Net Interest Income                                -1.823010e+08   
Interest Expense                                    1.514830e+08   
Interest Income                                     1.167000e+06   
Normalized Income                                  -2.100667e+08   
Net Income From Continuing And Discontinued Ope... -2.278100e+08   
Diluted NI Availto Com Stockholders                -2.278100e+08   
Net Income Common Stockholders                     -2.278100e+08   
Net Income                                         -2.278100e+08   
Net Income Including Noncontrolling Interests      -2.192430e+08   
Net Income Continuous Operations                   -2.192430e+08   
Pretax Income                                      -2.332340e+08   
Selling General And Administration                  3.384140e+08   
Total Revenue                                       1.085009e+09   
Operating Revenue                             

In [5]:
url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )
fundsSI[fundsSI['ROA']>0]

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
1,ABCB4,22.20,6.87,6.51,0.92,0.09,20.67,11.16,12.36,7.22,69.41,62.19,7.95,0.81,NaN,-0.09,NaN,14.16,1.38,1.25,0.10,0.90,0.11,20.08,14.89,1.296867e+07,24.07,3.41,-0.19,5.431382e+09
2,ABEV3,12.00,6.09,13.14,1.99,1.32,50.83,23.88,17.85,9.82,9.20,-0.62,-0.12,2.35,51.13,-1.80,1.11,15.11,10.04,17.45,0.66,0.33,0.56,9.68,5.54,3.158893e+08,6.05,0.91,15.89,1.890919e+11
6,AFLT3,7.34,1.73,16.46,1.77,1.55,66.84,60.77,52.68,14.27,13.85,-0.42,-0.05,8.67,9.06,-2.00,4.16,10.76,9.42,10.22,0.88,0.12,0.18,14.85,4.46,3.331640e+03,4.15,0.45,2.03,4.630417e+08
7,AGRO3,25.70,12.50,11.15,1.35,0.79,27.98,15.46,21.58,15.56,20.50,4.95,0.43,2.41,4.24,-1.24,2.04,12.09,7.08,5.06,0.59,0.41,0.33,27.22,13.38,6.290891e+06,19.06,2.31,0.04,2.638965e+09
9,AHEB3,31.00,NaN,2.49,2.33,0.52,31.24,21.73,16.68,1.91,1.11,-0.79,-0.96,0.42,1.59,-1.28,2.20,93.37,20.69,98.59,0.22,0.78,1.24,50.82,NaN,5.877890e+03,13.32,12.43,0.04,2.632557e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,WHRL4,4.42,8.47,17.14,2.97,0.74,16.37,11.01,3.34,5.20,4.70,-0.38,-0.22,0.57,25.57,-2.40,1.04,17.31,4.32,32.24,0.25,0.75,1.29,13.15,16.50,3.351581e+04,1.49,0.26,2.48,6.488009e+09
619,WIZC3,5.83,4.22,5.65,0.77,0.36,62.00,42.04,14.33,1.92,1.88,-0.04,-0.02,0.81,14.72,-0.47,1.11,13.58,6.31,22.57,0.46,0.54,0.44,13.90,-1.46,2.004735e+06,7.61,1.03,0.12,9.322595e+08
620,WLMM3,26.50,2.50,8.22,1.32,1.09,11.50,6.34,4.28,5.55,5.93,-0.17,-0.04,0.35,2.33,-2.55,5.56,16.12,13.30,16.62,0.82,0.17,3.11,25.90,48.84,9.420750e+03,20.00,3.22,-1.96,1.060436e+09
621,WLMM4,31.31,2.33,9.71,1.57,1.29,11.50,6.34,4.28,6.56,5.93,-0.17,-0.04,0.42,2.76,-3.01,5.56,16.12,13.30,16.62,0.82,0.17,3.11,25.90,48.84,2.557688e+04,20.00,3.22,-2.31,1.060436e+09
